# STEP 02. 봉차트

In [1]:
# 데이터 분석을 위해 pandas, numpy, 시각화를 위해 matplotlib 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pymysql.cursors

In [2]:
# 그래프 스타일 설정
plt.style.use("seaborn")

C:\Users\yyseo\AppData\Local\Temp\ipykernel_12568\4282763211.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


### 1. 개별 종목 일별 시세 가져오기

In [3]:
# 데이터베이스 연결
connection = pymysql.connect(host='localhost',   #데이터베이스 아이피
                             user='USER_SQLSTK',  #데이터베이스 사용자
                             password='1qaz2wsx', #데이터베이스 암호
                             database='DB_SQLSTK', #데이터베이스 이름
                             charset='utf8',       # 언어 설정 
                             cursorclass=pymysql.cursors.DictCursor # 조회결과를 dictionary로 리턴
                             )

In [4]:
cursor = connection.cursor()

In [5]:
#실행할 SQL
# 2019년 01월 01일 ~ 2020년 03월 19일 대상홀딩스의 종가 거래량 조회
sql = """
      select t1.STK_CD as 코드, t1.STK_NM as 종목,
             t2.DT as 날짜, t2.O_PRC as 시가, t2.C_PRC as 종가, t2.L_PRC as 저가, t2.H_PRC as 고가, t2.VOL as 거래량
      from stock as t1
           inner join history_dt as t2
                 on (t2.STK_CD = t1.STK_CD)
      where t1.STK_NM = '대상홀딩스'
            and t2.DT >= str_to_date('20190101', '%Y%m%d')
            and t2.DT <= str_to_date('20200319', '%Y%m%d')
"""  

In [6]:
# SQL 쿼리를 실행할 준비를 함
cursor.execute(sql)

300

In [7]:
# SQL 쿼리를 실행하고 결과를 result에 저장
result = cursor.fetchall()

In [8]:
# SQL 쿼리 실행 결과 출력
result

[{'코드': '084690',
  '종목': '대상홀딩스',
  '날짜': datetime.date(2019, 1, 2),
  '시가': Decimal('7850.000'),
  '종가': Decimal('7860.000'),
  '저가': Decimal('7600.000'),
  '고가': Decimal('7870.000'),
  '거래량': Decimal('17063.000')},
 {'코드': '084690',
  '종목': '대상홀딩스',
  '날짜': datetime.date(2019, 1, 3),
  '시가': Decimal('7860.000'),
  '종가': Decimal('7810.000'),
  '저가': Decimal('7590.000'),
  '고가': Decimal('7860.000'),
  '거래량': Decimal('57619.000')},
 {'코드': '084690',
  '종목': '대상홀딩스',
  '날짜': datetime.date(2019, 1, 4),
  '시가': Decimal('7800.000'),
  '종가': Decimal('8010.000'),
  '저가': Decimal('7600.000'),
  '고가': Decimal('8140.000'),
  '거래량': Decimal('53079.000')},
 {'코드': '084690',
  '종목': '대상홀딩스',
  '날짜': datetime.date(2019, 1, 7),
  '시가': Decimal('8220.000'),
  '종가': Decimal('8150.000'),
  '저가': Decimal('7800.000'),
  '고가': Decimal('8220.000'),
  '거래량': Decimal('116343.000')},
 {'코드': '084690',
  '종목': '대상홀딩스',
  '날짜': datetime.date(2019, 1, 8),
  '시가': Decimal('8100.000'),
  '종가': Decimal('8000.000'),

In [9]:
# result를 DataFrame으로 변환
df = pd.DataFrame(result)

In [10]:
df

,코드,종목,날짜,시가,종가,저가,고가,거래량
0,084690,대상홀딩스,2019-01-02,7850.000,7860.000,7600.000,7870.000,17063.000
1,084690,대상홀딩스,2019-01-03,7860.000,7810.000,7590.000,7860.000,57619.000
2,084690,대상홀딩스,2019-01-04,7800.000,8010.000,7600.000,8140.000,53079.000
3,084690,대상홀딩스,2019-01-07,8220.000,8150.000,7800.000,8220.000,116343.000
4,084690,대상홀딩스,2019-01-08,8100.000,8000.000,7800.000,8150.000,152998.000
...,...,...,...,...,...,...,...,...
295,084690,대상홀딩스,2020-03-13,4550.000,4355.000,4125.000,4550.000,155884.000
296,084690,대상홀딩스,2020-03-16,4285.000,4205.000,4205.000,4500.000,164420.000
297,084690,대상홀딩스,2020-03-17,4050.000,4080.000,3880.000,4150.000,140526.000
298,084690,대상홀딩스,2020-03-18,4050.000,3915.000,3915.000,4135.000,89795.000


In [11]:
#조회한 컬럼 정보 조회
cursor.description

(('코드', 253, None, 40, 40, 0, False),
 ('종목', 253, None, 200, 200, 0, True),
 ('날짜', 10, None, 10, 10, 0, False),
 ('시가', 246, None, 20, 20, 3, True),
 ('종가', 246, None, 20, 20, 3, True),
 ('저가', 246, None, 20, 20, 3, True),
 ('고가', 246, None, 20, 20, 3, True),
 ('거래량', 246, None, 20, 20, 3, True))

In [12]:
#조회한 컬럼의 정보를 desc에 저장
desc = cursor.description

In [13]:
desc

(('코드', 253, None, 40, 40, 0, False),
 ('종목', 253, None, 200, 200, 0, True),
 ('날짜', 10, None, 10, 10, 0, False),
 ('시가', 246, None, 20, 20, 3, True),
 ('종가', 246, None, 20, 20, 3, True),
 ('저가', 246, None, 20, 20, 3, True),
 ('고가', 246, None, 20, 20, 3, True),
 ('거래량', 246, None, 20, 20, 3, True))

In [14]:
#리스트의 행 열 방향을 반대로 바꿈
list( zip(*desc) )

[('코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량'),
 (253, 253, 10, 246, 246, 246, 246, 246),
 (None, None, None, None, None, None, None, None),
 (40, 200, 10, 20, 20, 20, 20, 20),
 (40, 200, 10, 20, 20, 20, 20, 20),
 (0, 0, 0, 3, 3, 3, 3, 3),
 (False, True, False, True, True, True, True, True)]

In [15]:
#리스트의 0번째 열 리턴 => 조회한 컬럼 이름
list( zip(*desc) )[0]

('코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량')

In [16]:
#조회한 컬럼 이름을 column_name 에 대입
column_name = list( zip(*desc) )[0]

In [17]:
column_name

('코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량')

In [18]:
# for element in column_name : column_name에서 컬럼 이름 1개를 element에 대입
#element.upper() : 컬럼 이름 1개가 저장된 element를 대문자로 변환해서 리턴

[element.upper() for element in column_name]

['코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량']

In [19]:
# for element in column_name : column_name에서 컬럼 이름 1개를 element에 대입
#element.upper() : 컬럼 이름 1개가 저장된 element를 대문자로 변환해서 리턴
#대문자로 변환한 컬럼 이름을 column_name에 대입

column_name = [element.upper() for element in column_name]

In [20]:
column_name

['코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량']

In [21]:
#re.sub() : 특정 문자를 찾아서 삭제
#\S : 문자열
#\. : .
# .앞의 문자열 찾아서 삭제

# T3.FIN_ITM_VAL -> FIN_ITM_VAL 으로 변환
re.sub('\S*\.',"","T3.FIN_ITM_VAL" ) 

'FIN_ITM_VAL'

In [22]:
#re.sub() : 특정 문자를 찾아서 삭제
#\S : 문자열
#\. : .
# .앞의 문자열 찾아서 삭제

# 컬럼 이름에 . 앞의 문자열 삭제

column_name = [re.sub('\S*\.',"",element)  for element in column_name]

In [23]:
column_name

['코드', '종목', '날짜', '시가', '종가', '저가', '고가', '거래량']

In [24]:
#컬럼 이름 수정
df.columns = column_name

In [25]:
df

,코드,종목,날짜,시가,종가,저가,고가,거래량
0,084690,대상홀딩스,2019-01-02,7850.000,7860.000,7600.000,7870.000,17063.000
1,084690,대상홀딩스,2019-01-03,7860.000,7810.000,7590.000,7860.000,57619.000
2,084690,대상홀딩스,2019-01-04,7800.000,8010.000,7600.000,8140.000,53079.000
3,084690,대상홀딩스,2019-01-07,8220.000,8150.000,7800.000,8220.000,116343.000
4,084690,대상홀딩스,2019-01-08,8100.000,8000.000,7800.000,8150.000,152998.000
...,...,...,...,...,...,...,...,...
295,084690,대상홀딩스,2020-03-13,4550.000,4355.000,4125.000,4550.000,155884.000
296,084690,대상홀딩스,2020-03-16,4285.000,4205.000,4205.000,4500.000,164420.000
297,084690,대상홀딩스,2020-03-17,4050.000,4080.000,3880.000,4150.000,140526.000
298,084690,대상홀딩스,2020-03-18,4050.000,3915.000,3915.000,4135.000,89795.000


In [26]:
# df의 컬럼 타입 조회
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   코드      300 non-null    object
 1   종목      300 non-null    object
 2   날짜      300 non-null    object
 3   시가      300 non-null    object
 4   종가      300 non-null    object
 5   저가      300 non-null    object
 6   고가      300 non-null    object
 7   거래량     300 non-null    object
dtypes: object(8)
memory usage: 18.9+ KB


In [27]:
df

,코드,종목,날짜,시가,종가,저가,고가,거래량
0,084690,대상홀딩스,2019-01-02,7850.000,7860.000,7600.000,7870.000,17063.000
1,084690,대상홀딩스,2019-01-03,7860.000,7810.000,7590.000,7860.000,57619.000
2,084690,대상홀딩스,2019-01-04,7800.000,8010.000,7600.000,8140.000,53079.000
3,084690,대상홀딩스,2019-01-07,8220.000,8150.000,7800.000,8220.000,116343.000
4,084690,대상홀딩스,2019-01-08,8100.000,8000.000,7800.000,8150.000,152998.000
...,...,...,...,...,...,...,...,...
295,084690,대상홀딩스,2020-03-13,4550.000,4355.000,4125.000,4550.000,155884.000
296,084690,대상홀딩스,2020-03-16,4285.000,4205.000,4205.000,4500.000,164420.000
297,084690,대상홀딩스,2020-03-17,4050.000,4080.000,3880.000,4150.000,140526.000
298,084690,대상홀딩스,2020-03-18,4050.000,3915.000,3915.000,4135.000,89795.000


In [28]:
# DT 컬럼의 타입을 날짜와 시간으로 변환
df["날짜"] = pd.to_datetime (df["날짜"])

In [29]:
# C_PRC 컬럼 부터 마지막 컬럼까지의 타입을 정수로 변환
df.loc[: , "종가":  ] = df.loc[: , "종가" : ].astype("int64")

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   코드      300 non-null    object        
 1   종목      300 non-null    object        
 2   날짜      300 non-null    datetime64[ns]
 3   시가      300 non-null    object        
 4   종가      300 non-null    object        
 5   저가      300 non-null    object        
 6   고가      300 non-null    object        
 7   거래량     300 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 18.9+ KB


In [32]:
#봉차트 그리기

candlestick = go.Candlestick(
                            x=df["날짜"], #X축 날짜
                            open=df["시가"], #시가
                            high=df["고가"], #고가
                            low=df["저가"], #저가
                            close=df["종가"], #종가
                            increasing_line_color= 'red', #양봉 색 설정
                            decreasing_line_color= 'blue' #음봉 색 설정
                            )

fig = go.Figure(data=[candlestick]) #봉차트를 fig에 저장

fig.update_layout(
    title = "2019.01.01~2020.03.19 대상홀딩스 주식 가격", #차트 제목 설정
    yaxis_title='주식 가격', #y축 제목 설정
)

fig.show() #차트 출력

In [34]:
#봉차트 그리기
candlestick = go.Candlestick(
                            x=df["날짜"], #X축 날짜
                            open=df["시가"], #시가
                            high=df["고가"], #고가
                            low=df["저가"], #저가
                            close=df["종가"], #종가
                            increasing_line_color= 'red', #양봉 색 설정
                            decreasing_line_color= 'blue', #음봉 색 설정
                            showlegend=False #범례를 출력하지 않음
                            )

#거래량 막대 그래프
volume_bars = go.Bar(
                    x=df["날짜"], #X좌표 날짜
                    y=df["거래량"], #y좌표거래량
                    showlegend=False, #범례를 출력하지 않음
                    marker={
                        "color": "rgba(128,128,128,0.5)", #거래량 막대 차트 색 -> 회색
                    }
                )

#2줄 2칸으로 나눠서 그래프를 그림
fig = make_subplots(rows=2,  # 줄
                    cols=1,  #칸
                    shared_xaxes=True, #x축 공유
                    subplot_titles=("2019.01.01~2020.03.19 대상홀딩스 주식 가격", '거래량') #그래프 제목 설정
                   )


# 1번째 줄 1번째 칸에 봉차트 출력
fig.add_trace(candlestick, row=1, col=1 )

# 2번째 줄 1번째 칸에 거래량 막대 차트 출력
fig.add_trace(volume_bars, row=2, col=1)


fig.show() #차트 출력